In [3]:
import xarray as xr
import numpy as np
import glob
import intake
import dask
from numcodecs import Blosc

In [4]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")
print(list(cat.FESOM))

['IFS_4.4-FESOM_5-cycle3']


In [5]:
run = cat.FESOM['IFS_4.4-FESOM_5-cycle3']
print(list(run))

['2D_1h_0.25deg', '2D_1h_1deg', '2D_1h_native', '2D_daily_0.25deg', '2D_daily_1deg', '2D_daily_native', '2D_monthly_0.25deg', '2D_monthly_1deg', '2D_monthly_native', '3D_1h_native_zarr', '3D_3h_native', '3D_daily_native', '3D_daily_native_zarr', 'elem_grid', 'node_grid']


In [ ]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
dask.config.set({'temporary_directory': '/scratch/a/a270088/dask/'})
# client = Client(n_workers=5, threads_per_worker=1, memory_limit='50GB')
client = Client(n_workers=20, threads_per_worker=1, memory_limit='10GB')
client

## Compute TS monthly means

In [6]:
# with dask.config.set(array_slicing_split_large_chunks=True):
data = run['3D_daily_native'].to_dask()

In [7]:
data

<xarray.Dataset>
Dimensions:  (nz: 70, time: 1808, elem: 14741520, nod2: 7402886, nz1: 69)
Coordinates:
  * nz       (nz) float64 0.0 5.0 10.0 15.0 ... 5.4e+03 5.65e+03 6e+03 6.35e+03
  * time     (time) datetime64[ns] 2020-01-20T23:56:00 ... 2024-12-31T23:56:00
  * nz1      (nz1) float64 2.5 7.5 12.5 17.5 ... 5.525e+03 5.825e+03 6.175e+03
Dimensions without coordinates: elem, nod2
Data variables:
    Av       (time, nz, elem) float32 dask.array<chunksize=(1, 70, 14741520), meta=np.ndarray>
    Kv       (time, nz, nod2) float32 dask.array<chunksize=(1, 70, 7402886), meta=np.ndarray>
    salt     (time, nz1, nod2) float32 dask.array<chunksize=(1, 69, 7402886), meta=np.ndarray>
    temp     (time, nz1, nod2) float32 dask.array<chunksize=(1, 69, 7402886), meta=np.ndarray>
    u        (time, nz1, elem) float32 dask.array<chunksize=(1, 69, 14741520), meta=np.ndarray>
    v        (time, nz1, elem) float32 dask.array<chunksize=(1, 69, 14741520), meta=np.ndarray>
    w        (time, nz, nod2) float32 dask.array<chunksize=(1, 70, 7402886), meta=np.ndarray>
Attributes: (12/17)
    FESOM_model:                         FESOM2
    FESOM_website:                       fesom.de
    FESOM_git_SHA:                       dfbf523
    FESOM_MeshPath:                      /work/bm1235/ifs-inputs_nxg_c3/fesom...
    FESOM_mesh_representative_checksum:  
    FESOM_ClimateDataPath:               /work/bm1235/ifs-inputs_nxg_c3/fesom...
    ...                                  ...
    FESOM_use_floatice:                  0
    FESOM_whichEVP:                      1
    FESOM_evp_rheol_steps:               120
    FESOM_opt_visc:                      5
    FESOM_use_wsplit:                    -1
    FESOM_autorotate_back_to_geo:        -1

In [16]:
for year in range(2020, 2022):
    print(year)
    for vari in ['u', 'v', 'w']:
        print(f"variable: {vari}, year: {year}")
        vari_mm = data[vari].sel(time=str(year)).resample(time='1M').mean(dim='time')
        vari_mm.to_netcdf(f'/work/bm1344/AWI/Cycle3/test/monthly_means/{vari}/{vari}_{str(year)}.nc')

2020
variable: salt, year: 2020
variable: temp, year: 2020
2021
variable: salt, year: 2021
variable: temp, year: 2021
